# 50.040 Natural Language Processing (Summer 2020) Homework 2

**Due**


### STUDNET ID:

### Name:

### Students with whom you have discussed (if any):

In [275]:
import copy
from collections import Counter
from nltk.tree import Tree
from nltk import Nonterminal
from nltk.corpus import LazyCorpusLoader, BracketParseCorpusReader
from collections import defaultdict
import time

In [276]:
st = time.time()

In [277]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to C:\Users\Student-
[nltk_data]     TeoYongQuan\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [278]:
def set_leave_lower(tree_string):
    if isinstance(tree_string, Tree):
        tree = tree_string
    else:
        tree = Tree.fromstring(tree_string)
    for idx, _ in enumerate(tree.leaves()):
        tree_location = tree.leaf_treeposition(idx)
        non_terminal = tree[tree_location[:-1]]
        non_terminal[0] = non_terminal[0].lower()
    return tree

def get_train_test_data():
    '''
    Load training and test set from nltk corpora
    '''
    train_num = 3900
    test_index = range(10)
    treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
    cnf_train = treebank.parsed_sents()[:train_num]
    cnf_test = [treebank.parsed_sents()[i+train_num] for i in test_index]
    #Convert to Chomsky norm form, remove auxiliary labels
    cnf_train = [convert2cnf(t) for t in cnf_train]
    cnf_test = [convert2cnf(t) for t in cnf_test]
    return cnf_train, cnf_test
def convert2cnf(original_tree):
    '''
    Chomsky norm form
    '''
    tree = copy.deepcopy(original_tree)
    
    #Remove cases like NP->DT, VP->NP
    tree.collapse_unary(collapsePOS=True, collapseRoot=True)
    #Convert to Chomsky
    tree.chomsky_normal_form()
    
    tree = set_leave_lower(tree)
    return tree

In [279]:
### GET TRAIN/TEST DATA
cnf_train, cnf_test = get_train_test_data()

In [280]:
cnf_train[0].pprint()

(S
  (NP-SBJ
    (NP (NNP pierre) (NNP vinken))
    (NP-SBJ|<,-ADJP-,>
      (, ,)
      (NP-SBJ|<ADJP-,>
        (ADJP (NP (CD 61) (NNS years)) (JJ old))
        (, ,))))
  (S|<VP-.>
    (VP
      (MD will)
      (VP
        (VB join)
        (VP|<NP-PP-CLR-NP-TMP>
          (NP (DT the) (NN board))
          (VP|<PP-CLR-NP-TMP>
            (PP-CLR
              (IN as)
              (NP
                (DT a)
                (NP|<JJ-NN> (JJ nonexecutive) (NN director))))
            (NP-TMP (NNP nov.) (CD 29))))))
    (. .)))


## Question 1

To  better  understand  PCFG,  let’s  consider  the  first  parse  tree  in  the training data “cnf_train” as an example.  Run the code we have provided for you and then writedown the roles of.productions(), .rhs(), .lhs(), .leaves()in the ipynb notebook.

In [281]:
rules = cnf_train[0].productions()
print(rules, type(rules[0]))

[S -> NP-SBJ S|<VP-.>, NP-SBJ -> NP NP-SBJ|<,-ADJP-,>, NP -> NNP NNP, NNP -> 'pierre', NNP -> 'vinken', NP-SBJ|<,-ADJP-,> -> , NP-SBJ|<ADJP-,>, , -> ',', NP-SBJ|<ADJP-,> -> ADJP ,, ADJP -> NP JJ, NP -> CD NNS, CD -> '61', NNS -> 'years', JJ -> 'old', , -> ',', S|<VP-.> -> VP ., VP -> MD VP, MD -> 'will', VP -> VB VP|<NP-PP-CLR-NP-TMP>, VB -> 'join', VP|<NP-PP-CLR-NP-TMP> -> NP VP|<PP-CLR-NP-TMP>, NP -> DT NN, DT -> 'the', NN -> 'board', VP|<PP-CLR-NP-TMP> -> PP-CLR NP-TMP, PP-CLR -> IN NP, IN -> 'as', NP -> DT NP|<JJ-NN>, DT -> 'a', NP|<JJ-NN> -> JJ NN, JJ -> 'nonexecutive', NN -> 'director', NP-TMP -> NNP CD, NNP -> 'nov.', CD -> '29', . -> '.'] <class 'nltk.grammar.Production'>


In [282]:
rules[0].rhs(), type(rules[0].rhs()[0])

((NP-SBJ, S|<VP-.>), nltk.grammar.Nonterminal)

In [283]:
rules[10].rhs(), type(rules[10].rhs()[0])

(('61',), str)

In [284]:
rules[0].lhs(), type(rules[0].lhs())

(S, nltk.grammar.Nonterminal)

In [285]:
print(cnf_train[0].leaves())

['pierre', 'vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', '29', '.']


ANSWER HERE
- productions(): Rykes tgat firn tge tree
- rhs(): right side childrens in the production rule
- lhs(): parents in the production rule
- leaves(): a list of string

## Question 2
To count the number of unique rules, nonterminals and terminals, pleaseimplement functions **collect_rules, collect_nonterminals, collect_terminals**


In [286]:
def collect_rules(train_data):
    '''
    Collect the rules that appear in data.
    params:
        train_data: list[Tree] --- list of Tree objects
    return:
        rules: list[nltk.grammar.Production] --- list of rules (Production objects)
        rules_counts: Counter object --- a dictionary that maps one rule (nltk.Nonterminal) to its number of 
                                         occurences (int) in train data.
    '''
    rules = list()
    rules_counts = Counter()
    ### YOUR CODE HERE (~ 2 lines)
    for i in train_data:
      for j in i.productions():
        rules.append(j)
    rule_counts = Counter(rules)
    
    ### YOUR CODE HERE
    return rules, rule_counts

def collect_nonterminals(rules):
    '''
    collect nonterminals that appear in the rules
    params:
        rules: list[nltk.grammar.Production] --- list of rules (Production objects)
    return:
        nonterminals: set(nltk.Nonterminal) --- set of nonterminals 
    '''
    nonterminals = list()
    ### YOUR CODE HERE (at least one line)
    for rule in rules:
      lhs = rule.lhs()
      rhs = rule.rhs()
      #for nt in lhs:
      if type(lhs) == nltk.Nonterminal:
        nonterminals.append(lhs)
      #for nt in rhs:
      if type(rhs) == nltk.Nonterminal:
        nonterminals.append(rhs)
    ### END OF YOUR CODE
    return set(nonterminals)

def collect_terminals(rules):
    '''
    collect terminals that appear in the rules
    params:
        rules: list[nltk.grammar.Production] --- list of rules (Production objects)
    return:
        terminals: set of strings --- set of terminals    
    '''
    terminals = list()
    ### YOUR CODE HERE (at least one line)
    for rule in rules:
      for terminal in rule.rhs():
        if type(terminal) == str:
          terminals.append(terminal)
    ### END OF YOUR CODE

    return set(terminals)

In [287]:
train_rules, train_rules_counts = collect_rules(cnf_train)
nonterminals = collect_nonterminals(train_rules)
terminals = collect_terminals(train_rules)

In [288]:
### CORRECT ANSWER (19xxxx, 3xxxx, 1xxxx, 7xxx)
len(train_rules), len(set(train_rules)), len(terminals), len(nonterminals)

(196646, 31656, 11367, 7869)

In [289]:
print(train_rules_counts.most_common(5))

[(, -> ',', 4876), (DT -> 'the', 4726), (. -> '.', 3814), (PP -> IN NP, 3273), (S|<VP-.> -> VP ., 3003)]


## Question 3
Implement the function **build_pcfg** which builds a dictionary that stores theterminal rules and nonterminal rules.

In [290]:
def build_pcfg(rules_counts):
    '''
    Build a dictionary that stores the terminal rules and nonterminal rules.
    param:
        rules_counts: Counter object --- a dictionary that maps one rule to its number of occurences in train data.
    return:
        rules_dict: dict(dict(dict)) --- a dictionary has a form like:
                    rules_dict = {'terminals':{'NP':{'the':1000,'an':500}, 'ADJ':{'nice':500,'good':100}},
                                  'nonterminals':{'S':{'NP@VP':1000},'NP':{'NP@NP':540}}}
    When building "rules_dict", you need to use "lhs()", "rhs()" funtion and convert Nonterminal to str.
    All the keys in the dictionary are of type str.
    '@' is used as a special symbol to split left and right nonterminal strings.
    '''
    
    rules_dict = dict()
    ### rules_dict['terminals'] contains rules like "NP->'the'"
    ### rules_dict['nonterminals'] contains rules like "S->NP@VP"
    rules_dict['terminals'] = defaultdict(dict)
    rules_dict['nonterminals'] = defaultdict(dict)
    
    ### YOUR CODE HERE
    i=0
    for rule,count in rules_counts.items():
        
        if type(rule.rhs()[0]) == str:
        #this is a terminal
            #print(str(rule.rhs()))
            rules_dict["terminals"][str(rule.lhs())] = rules_dict["terminals"].get(str(rule.lhs()),{}) 
            rules_dict["terminals"][str(rule.lhs())][str(rule.rhs()[0])] = rules_dict["terminals"][str(rule.lhs())].get(str(rule.rhs()[0]),0) + count
        else:
            string_right = ""
            for right in rule.rhs():
                string_right += str(right)
                string_right += "@"
            string_right = string_right[:-1]
            #if string_right == "NNP@NNP":
                #print(string_right,str(rule.lhs()))
            rules_dict["nonterminals"][str(rule.lhs())] = rules_dict["nonterminals"].get(str(rule.lhs()),{}) 
            rules_dict["nonterminals"][str(rule.lhs())][string_right] = rules_dict["nonterminals"][str(rule.lhs())].get(string_right,0) + count
    ### END OF YOUR CODE
    return rules_dict

In [291]:
train_rules_dict = build_pcfg(train_rules_counts)
#print(train_rules_dict)

## Question 4
Estimate the probability of rule $NP\rightarrow NNP@NNP$ 

In [292]:
count_np = sum(train_rules_dict['nonterminals']['NP'].values()) + sum(train_rules_dict['terminals']['NP'].values())
prob = train_rules_dict['nonterminals']['NP']['NNP@NNP'] / count_np

print("Estimated probability of \'NP -> NNP@NNP\' is", qMLE)

Estimated probability of 'NP -> NNP@NNP' is 0.03950843529348353


## Question 5
Find the terminal symbols in ''cnf_test[0]'' that never appeared in the PCFG we built. 

In [293]:
#print(cnf_test[0])
pcfg_terminals = {}
out= []
for i in train_rules_dict['terminals'].values():
    #print(i)
    for rhs in i.keys():
        pcfg_terminals[rhs] = 1
#print(pcfg_terminals)
for rule in cnf_test[0].leaves():
    if type(rule) == str:
        n = pcfg_terminals.get(rule,0)
        if n == 0:
            out.append(rule)
print("The symbols that did not appear are : ",out)
        

The symbols that did not appear are :  ['constitutional-law']


## Question 6
We can use smoothing techniques to handle these cases. A simple smoothing method is as follows. We first create a new "unknown" terminal symbol $unk$.

Next, for each original non-terminal symbol $A\in N$, we add one new rule $A \rightarrow unk$ to the original PCFG.

The smoothed probabilities for all rules can then be estimated as:
$$q_{smooth}(A \rightarrow \beta) = \frac {count(A \rightarrow \beta)}{count(A)+1}$$
$$q_{smooth}(A \rightarrow unk) = \frac {1}{count(A)+1}$$
where $|V|$ is the count of unique terminal symbols.


Implement the function **smooth_rules_prob** which returns the smoothed rule probabilities

In [294]:
def smooth_rules_prob(rules_counts):
    '''
    params:
        rules_counts: dict(dict(dict)) --- a dictionary has a form like:
                      rules_counts = {'terminals':{'NP':{'the':1000,'an':500}, 'ADJ':{'nice':500,'good':100}},
                                      'nonterminals':{'S':{'NP@VP':1000},'NP':{'NP@NP':540}}}
    
    return:
        rules_prob: dict(dict(dict)) --- a dictionary that has a form like:
                               rules_prob = {'terminals':{'NP':{'the':0.6,'an':0.3, '<unk>':0.1},
                                                          'ADJ':{'nice':0.6,'good':0.3,'<unk>':0.1},
                                                          'S':{'<unk>':0.01}}}
                                             'nonterminals':{'S':{'NP@VP':0.99}}
    '''
    rules_prob = copy.deepcopy(rules_counts)
    unk = '<unk>'
    ### Hint: don't forget to consider nonterminal symbols that don't appear in rules_counts['terminals'].keys()
    ### YOUR CODE HERE
    for term in (rules_prob['terminals'].keys()):
        #add unk
        rules_prob['terminals'][term][unk] = 1
        #get total counts
        count = 0
        count += sum(rules_prob['nonterminals'][term].values())
        count += sum(rules_prob['terminals'][term].values())
        # set the probability
        for key,val in rules_prob['terminals'][term].items():
            #print(leaf,count)
            rules_prob['terminals'][term][key] = val/(count+1)
    #get nonterminal symbols that dont appear in terminals
    missing_dict = {}
    for i in rules_prob['nonterminals'].keys():
        #print(i)
        if i not in rules_prob['terminals'].keys():
            missing_dict[i] = 1
    
    #for each non terminal
    for nonterm in rules_prob["nonterminals"].keys():
        count = 0
        count += sum(rules_prob['nonterminals'][nonterm].values())
        count += sum(rules_prob['terminals'][nonterm].values())
        for key,val in rules_prob['nonterminals'][nonterm].items():
            #print(leaf,count)
            rules_prob['nonterminals'][nonterm][key] = val/(count+1)
        if nonterm in missing_dict.keys():
            rules_prob["terminals"][nonterm][unk] = 1/(count+1)

    
        
            
    ### END OF YOUR CODE
    return rules_prob

In [295]:
s_rules_prob = smooth_rules_prob(train_rules_dict)
terminals.add('<unk>')

In [296]:
print(s_rules_prob['nonterminals']['S']['NP-SBJ@S|<VP-.>'])
print(s_rules_prob['nonterminals']['S']['NP-SBJ-1@S|<VP-.>'])
print(s_rules_prob['nonterminals']['NP']['NNP@NNP'])
print(s_rules_prob['terminals']['NP'])

0.1300172371337109
0.025240088648116228
0.03950630580310736
{'<unk>': 5.3893829156561574e-05}


In [297]:
len(terminals)

11368

## CKY Algorithm

Similar to the Viterbi algorithm, the CKY algorithm is a dynamic-programming algorithm. Given a PCFG $G=(N, \ \Sigma, \ S, \ R, \ q)$, we can use the CKY algorithm described in class to find the highest scoring parse tree for a sentence. 

First, let us complete the *CKY* function from scratch using only Python built-in functions and the Numpy package. 

The output should be two dictionaries $\pi$ and $bp$, which store the optimal probability and backpointer information respectively.

Given a sentence $w_0, w_1, ...,w_{n-1}$,  $\pi(i, k, X)$, $bp(i, k, X)$ refer to the highest score and backpointer for the (partial) parse tree that has the root X (a non-terminal symbol) and covers the word span $w_i, ..., w_{k-1}$, where $0 \le i < k \le n$. Note that a backpointer includes both the best grammar rule chosen and the best split point.
![tree](imgs/parse_tree.png)


## Question 7
Implement **CKY** function and run the test code to check your implementation.

In [298]:
def CKY(sent, rules_prob):
    '''
    params:
        sent: list[str] --- a list of strings
        rules_prob: dict(dict(dict)) --- a dictionary that has a form like:
                                           rules_prob = {'terminals':{'NP':{'the':0.6,'an':0.3, '<unk>':0.1},
                                                                      'ADJ':{'nice':0.6,'good':0.3,'<unk>':0.1},
                                                                      'S':{'<unk>':0.01}}}
                                                         'nonterminals':{'S':{'NP@VP':0.99}}
    return:
        score: dict() --- score[(i,i+span)][root] represents the highest score for the parse (sub)tree that has the root "root"
                          across words w_i, w_{i+1},..., w_{i+span-1}.
        back: dict() --- back[(i,i+span)][root] = (split , left_child, right_child); split: int; 
                         left_child: str; right_child: str. 
    '''
    score = defaultdict(dict)
    back = defaultdict(dict)
    sent_len = len(sent)
    ### YOUR CODE HERE
    nonterm = rules_prob["terminals"].keys()
    store = defaultdict(list)
    for i in range(sent_len):
        for A in nonterm:
            if rules_prob["terminals"][A].get(sent[i],0) != 0:
                score[(i, i+1)][A] = np.log(rules_prob['terminals'][A][sent[i]])
                back[(i, i+1)][A] = (i, sent[i], None)
                
    
                
    for span in range(2,sent_len+1):
        for begin in range(0,sent_len+1-span):
            end = begin+span
            for split in range(begin+1,end):
                for A in rules_prob['nonterminals'].keys():
                        for BC, prob in rules_prob['nonterminals'][A].items():
                            B, C = BC.split('@')
                            if B in back[(begin,split)].keys() and C in back[(split,end)].keys():
                                left = score[(begin,split)].get(B,0)
                                right = score[(split,end)].get(C,0)
                                cur_score = left+right+np.log(prob)
                                if cur_score > score[(begin,end)].get(A,-999999):
                                    score[(begin,end)][A] = cur_score
                                    back[(begin,end)][A] = (split,B,C)
    

            
        
    ### END OF YOUR CODE                  
    return score, back

In [299]:
sent = cnf_train[0].leaves()
score, back = CKY(sent, s_rules_prob)

In [300]:
score[(0, len(sent))]['S']

-117.52653989576537

## Question 8
Implement **build_tree** function according to algorithm 2 to reconstruct theparse tree

In [301]:
def build_tree(back, root,start):
    '''
    Build the tree recursively.
    params:
        back: dict() --- back[(i,i+span)][X] = (split , left_child, right_child); split:int; left_child: str; right_child: str.
        root: tuple() --- (begin, end, nonterminal_symbol), e.g., (0, 10, 'S
    return:
        tree: nltk.tree.Tree
    '''
    begin = root[0]
    end = root[1]
    root_label = root[2]
    ### YOUR CODE HERE
    split,left,right = back[(begin,end)][root_label]
    
    if right == None:
        tree = nltk.tree.Tree(root_label,[left])
    else: 
        left_tree = build_tree(back,(begin,split,left),start)
        right_tree = build_tree(back,(split,end,right),start)
        tree = nltk.tree.Tree(root_label,[left_tree,right_tree])
    

    ### END OF YOUR CODE 
    return tree

In [302]:
build_tree(back, (0, len(sent), 'S'), nonterminals).pprint()

(S
  (NP-SBJ
    (NP (NNP pierre) (NNP vinken))
    (NP-SBJ|<,-NP-,>
      (, ,)
      (NP-SBJ|<NP-,>
        (NP (CD 61) (NP|<NNS-JJ> (NNS years) (JJ old)))
        (, ,))))
  (S|<VP-.>
    (VP
      (MD will)
      (VP
        (VB join)
        (VP|<NP-PP-CLR-NP-TMP>
          (NP (DT the) (NN board))
          (VP|<PP-CLR-NP-TMP>
            (PP-CLR
              (IN as)
              (NP
                (DT a)
                (NP|<JJ-NN> (JJ nonexecutive) (NN director))))
            (NP-TMP (NNP nov.) (CD 29))))))
    (. .)))


## Question 9

In [303]:
def set_leave_index(tree):
    '''
    Label the leaves of the tree with indexes
    Arg:
        tree: original tree, nltk.tree.Tree
    Return:
        tree: preprocessed tree, nltk.tree.Tree
    '''
    for idx, _ in enumerate(tree.leaves()):
        tree_location = tree.leaf_treeposition(idx)
        non_terminal = tree[tree_location[:-1]]
        non_terminal[0] = non_terminal[0] + "_" + str(idx)
    return tree

def get_nonterminal_bracket(tree):
    '''
    Obtain the constituent brackets of a tree
    Arg:
        tree: tree, nltk.tree.Tree
    Return:
        nonterminal_brackets: constituent brackets, set
    '''
    nonterminal_brackets = set()
    for tr in tree.subtrees():
        label = tr.label()
        #print(tr.leaves())
        if len(tr.leaves()) == 0:
            continue
        start = tr.leaves()[0].split('_')[-1]
        end = tr.leaves()[-1].split('_')[-1]
        if start != end:
            nonterminal_brackets.add(label+'-('+start+':'+end+')')
    return nonterminal_brackets

def word2lower(w, terminals):
    '''
    Map an unknow word to "unk"
    '''
    return w.lower() if w in terminals else '<unk>'

In [304]:
correct_count = 0
pred_count = 0
gold_count = 0
for i, t in enumerate(cnf_test):
    #Protect the original tree 
    t = copy.deepcopy(t)
    sent = t.leaves()  
    #Map the unknow words to "unk"
    sent = [word2lower(w.lower(), terminals) for w in sent]
    
    #CKY algorithm
    score, back = CKY(sent, s_rules_prob)
    candidate_tree = build_tree(back, (0, len(sent), 'S'), nonterminals)
    
    #Extract constituents from the gold tree and predicted tree
    pred_tree = set_leave_index(candidate_tree)
    pred_brackets = get_nonterminal_bracket(pred_tree)
    
    #Count correct constituents
    pred_count += len(pred_brackets)
    gold_tree = set_leave_index(t)
    gold_brackets = get_nonterminal_bracket(gold_tree)
    gold_count += len(gold_brackets)
    current_correct_num = len(pred_brackets.intersection(gold_brackets))
    correct_count += current_correct_num
    
    print('#'*20)
    print('Test Tree:', i+1)
    print('Constituent number in the predicted tree:', len(pred_brackets))
    print('Constituent number in the gold tree:', len(gold_brackets))
    print('Correct constituent number:', current_correct_num)

recall = correct_count/gold_count
precision = correct_count/pred_count
f1 = 2*recall*precision/(recall+precision)

####################
Test Tree: 1
Constituent number in the predicted tree: 20
Constituent number in the gold tree: 20
Correct constituent number: 14
####################
Test Tree: 2
Constituent number in the predicted tree: 54
Constituent number in the gold tree: 54
Correct constituent number: 27
####################
Test Tree: 3
Constituent number in the predicted tree: 30
Constituent number in the gold tree: 30
Correct constituent number: 23
####################
Test Tree: 4
Constituent number in the predicted tree: 17
Constituent number in the gold tree: 17
Correct constituent number: 16
####################
Test Tree: 5
Constituent number in the predicted tree: 32
Constituent number in the gold tree: 32
Correct constituent number: 26
####################
Test Tree: 6
Constituent number in the predicted tree: 40
Constituent number in the gold tree: 40
Correct constituent number: 18
####################
Test Tree: 7
Constituent number in the predicted tree: 22
Constituent number in

In [305]:
print('Overall precision: {:.3f}, recall: {:.3f}, f1: {:.3f}'.format(precision, recall, f1))

Overall precision: 0.535, recall: 0.535, f1: 0.535


In [306]:
print('Overall precision: {:.3f}, recall: {:.3f}, f1: {:.3f}'.format(precision, recall, f1))

Overall precision: 0.535, recall: 0.535, f1: 0.535


In [307]:
et=time.time()
print(et - st)

912.660706281662
